# Imports

In [1]:
import pandas as pd
import numpy as np

# Init

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import torch.nn.init as init


# Load data
data = pd.read_csv("preprocessed_hourly_data.csv")
target_column = 'Close'

# Use all columns except datetime and target as features
features = [col for col in data.columns if col not in ['datetime', target_column]]

# Chronological split
train_size = int(0.65 * len(data))
val_size = int(0.15 * len(data))
test_size = len(data) - train_size - val_size

train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size+val_size]
test_data = data.iloc[train_size+val_size:]

# Sequence creation
def create_sequences(features, target, seq_length):
    X, y = [], []
    for i in range(len(features) - seq_length):
        X.append(features[i:i+seq_length])
        y.append(target[i+seq_length])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

seq_length = 24
X_train, y_train = create_sequences(train_data[features].values, train_data[target_column].values, seq_length)
X_val, y_val = create_sequences(val_data[features].values, val_data[target_column].values, seq_length)
X_test, y_test = create_sequences(test_data[features].values, test_data[target_column].values, seq_length)

# Convert to tensors
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

# DataLoader setup
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)


class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=0.2
        )
        self.fc = nn.Linear(hidden_size, output_size)
        
        # Initialize weights properly
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.ones_(param)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.ones_(self.fc.bias)
    
    def forward(self, x):
        # x shape: [batch, seq_len, features]
        lstm_out, _ = self.lstm(x)
        # Use only the last output
        return self.fc(lstm_out[:, -1, :])




# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleLSTM(input_size=len(features)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.ReduceLROnPlateau(optimizer, patience=5)
criterion = nn.HuberLoss()

# Training loop
best_val_loss = float('inf')
for epoch in range(10):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item() * len(X_batch)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch).squeeze()
            val_loss += criterion(outputs, y_batch).item() * len(X_batch)
    
    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    scheduler.step(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
    
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")

# Evaluation Fix
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

predictions = []
actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy().flatten()
        predictions.extend(outputs)
        actuals.extend(y_batch.numpy().flatten())

# Critical Fix: Proper Alignment
test_prices = data[target_column].values[train_size+val_size:]
naive_pred = test_prices[:-1]  # Previous prices
predictions = predictions[1:]  # Align with naive baseline
actuals = actuals[1:]  # Remove first prediction

# Final Length Check
min_length = min(len(actuals), len(predictions), len(naive_pred))
actuals = actuals[:min_length]
predictions = predictions[:min_length]
naive_pred = naive_pred[:min_length]

# Metrics
mse = mean_squared_error(actuals, predictions)
huber = criterion(torch.tensor(predictions), torch.tensor(actuals)).item()
naive_mse = mean_squared_error(actuals, naive_pred)

print(f"\nTest MSE: {mse:.2f}")
print(f"Test Huber: {huber:.2f}")
print(f"Naive Baseline MSE: {naive_mse:.2f}")

# Plotting
plt.figure(figsize=(14,6))
plt.plot(actuals[:200], label='Actual Prices')
plt.plot(predictions[:200], label='Predicted Prices')
plt.plot(naive_pred[:200], label='Naive Baseline', alpha=0.7)
plt.legend()
plt.title("Price Predictions vs Actuals (First 200 Samples)")
plt.show()



Target column: Close
Epoch 1/50, Train Loss: 0.0001, Val Loss: 0.0005
Epoch 2/50, Train Loss: 0.0000, Val Loss: 0.0001
Epoch 3/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 4/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 5/50, Train Loss: 0.0000, Val Loss: 0.0001
Epoch 6/50, Train Loss: 0.0000, Val Loss: 0.0001
Epoch 7/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 8/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 9/50, Train Loss: 0.0000, Val Loss: 0.0001
Epoch 10/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 11/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 12/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 13/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 14/50, Train Loss: 0.0000, Val Loss: 0.0001
Epoch 15/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 16/50, Train Loss: 0.0000, Val Loss: 0.0001
Epoch 17/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 18/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 19/50, Train Loss: 0.0000, Val Loss: 0.0000
Epoch 20/50, Train Loss: 0.0000, Val L